In [32]:
%matplotlib inline
%load_ext watermark
%watermark -v -m -p numpy,sklearn,tensorflow

CPython 3.5.6
IPython 7.7.0

numpy 1.15.2
sklearn 0.20.0
tensorflow 1.14.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-60-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [33]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [34]:
sound_names = ["amino acid sample", "amino acid standard", "else"]
sound_data = np.load('urban_sound.npz')
X_test= sound_data['X']
y_test = sound_data['y']
X_test.shape, y_test.shape

((9, 193), (9, 3))

In [35]:
training_epochs = 100
n_dim = 193
n_classes = 3
learning_rate = 0.01

In [36]:
g1 = tf.Graph()
with g1.as_default():
    X = tf.placeholder(tf.float32, [None, n_dim])
    Y = tf.placeholder(tf.float32, [None, n_classes])

    c1 = tf.layers.conv2d(tf.reshape(X, [-1, 1, n_dim, 1]), 50, (1, 5), padding='same', 
                          activation=tf.nn.sigmoid, name="c1")
    p1 = tf.layers.max_pooling2d(inputs=c1, pool_size=[1, 2], strides=2)
    c2 = tf.layers.conv2d(tf.reshape(p1, [-1, 1, 96, 50]), 100, (1, 5), padding='same', 
                          activation=tf.nn.sigmoid, name="c2")
    p2 = tf.layers.max_pooling2d(inputs=c2, pool_size=[1, 2], strides=2)

    h_p = tf.reshape(p2, [-1, 48*100])

    h_1 = tf.layers.dense(inputs=h_p, units=4800, activation=tf.nn.sigmoid,
                          kernel_initializer=tf.contrib.layers.xavier_initializer(), name="fc1")

    y_hat = tf.layers.dense(inputs=h_1, units=n_classes,
                            kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                            name="h4")

    correct_prediction = tf.equal(tf.argmax(y_hat,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [37]:
with tf.Session(graph=g1) as sess:
    sess.run(init)
    saver.restore(sess, 'model_adam.ckpt')
    print('Test accuracy: ',round(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}) , 3))

ValueError: The passed save_path is not a valid checkpoint: model_adam.ckpt